In [ ]:
let input = System.IO.File.ReadLines("day7_input.txt")

In [ ]:
open System.Text.RegularExpressions

let (|Command|_|) (command:string) (string:string) =
    let pattern = "$ " + command
    if string.StartsWith(pattern) then
        Some(string.Substring(pattern.Length))
    else
        None

let (|DirItem|_|) (string:string) =
    let pattern = "dir "
    if string.StartsWith(pattern) then
        Some(string.Substring(4))
    else
        None

let (|FileItem|_|) (string:string) =
    let pattern = Regex("\d+ .+")
    if pattern.IsMatch string then
        match string.Split " " |> Array.toList with
        | [size; name] -> Some((name, int size))
        | _ -> failwith "Invalid file"
    else
        None

In [ ]:
open System.Collections.Generic

type Node =
    | Directory of string *  List<Node>
    | File of string * int

let is_dir name dir : bool = 
    match dir with
    | Directory(dir_name, _) when dir_name = name -> true
    | _ -> false

let rec findWithPath path directory =
    match directory with
    | File _ -> failwith "Path can't be a file"
    | Directory(dirname, content) ->
        match path with
        | [] -> directory
        | head::tail -> findWithPath tail (content.Find (is_dir head))

let skipLast list = 
    list
    |> List.rev
    |> List.skip 1
    |> List.rev

let insert state node =
    let current = findWithPath (fst state) (snd state)
    match current with
    | File(_, _) -> failwith "Current can't be a file"
    | Directory(_, list) -> 
        list.Add(node)
        state

let parseLine state line =
    match line with
    | Command "ls" _ -> state
    | Command "cd .." _ -> (state |> fst |> skipLast, snd state)
    | Command "cd " dir -> ((fst state) @ [dir], snd state)
    | DirItem name -> insert state (Directory(name, new List<Node>()))
    | FileItem (name, size) -> insert state (File(name, size))
    | _ -> failwith "Invalid line"

let rec listDirRec acc node =
    match node with
    | Directory(_, content) ->
        content
        |> List.ofSeq
        |> List.fold listDirRec ([node] @ acc)
    | _ -> acc

let listDir node = 
    match node with
    | Directory(_, _) -> listDirRec [] node
    | _ -> failwith "invalid node"


In [ ]:
let rec dirSize node =
    match node with
    | File(_,_) -> failwith "No file allowed"
    | Directory(_, content) ->
        content
        |> List.ofSeq
        |> List.fold (fun acc node -> acc + (sizeOf node)) 0

and sizeOf node =
    match node with
    | File(_, size) -> size
    | Directory(_,_) -> dirSize node

In [ ]:
let initial_state = ([], Directory("/", new List<Node>()))

input
|> Seq.skip 1
|> Seq.fold parseLine initial_state

let (_, root) = initial_state

let dir_sizes =
    root
    |> listDir
    |> List.fold (fun acc dir ->[(dirSize dir)] @ acc) []

dir_sizes
|> List.filter ((>) 100000)
|> List.sum
|> printfn "Part 1 : %i"

let total = 70000000
let required = 30000000 - (total - (dirSize root))

dir_sizes
|> List.sort
|> List.find ((<) required)
|> printfn "Part 2 : %i" 